In [1]:
!pip install pathway bokeh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [7]:
df['Timestamp']= pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'] , format="%d-%m-%Y %H:%M:%S")

In [11]:
# vehicle type to weight conversion .
vehicle_weight = {'bike': 0.5 ,'car':1.0,'truck':1.5}
df['VehicleWeight']=df['VehicleType'].map(vehicle_weight)

In [12]:
# traffic level to weight conversion .
traffic_map={'low': 1,'medium':2, 'high':3}
df['TrafficLevel']=df['TrafficConditionNearby'].map(traffic_map)

In [10]:
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [13]:
#MODEL 1 .
def model1_linear(df , base_price=10.0,alpha=2.0):
  df =df.copy()
  df['Price_Model1']= base_price
  prev_price = {}
  for i , row in df.iterrows():
    lot =row['SystemCodeNumber']
    occ_ratio =row['Occupancy']/row['Capacity']

    if lot not in prev_price:
      prev_price[lot]=base_price

    price =prev_price[lot] + alpha*occ_ratio
    df.at[i,'Price_Model1'] = price
    prev_price[lot]=price
  return df
df_model1 = model1_linear(df)

In [14]:
#MODEL2
def model2_demand_based(df, base_price=10.0,
                        alpha=1.2, beta=0.8, gamma=0.5,
                        delta=1.0, epsilon=0.7, lambd=0.3):
    df = df.copy()

    df['RawDemand'] = (
        alpha * (df['Occupancy'] / df['Capacity']) +
        beta * df['QueueLength'] -
        gamma * df['TrafficLevel'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleWeight']
    )


    df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min() + 1e-5))


    df['Price_Model2'] = base_price * (1 + lambd * df['NormalizedDemand'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=base_price * 0.5, upper=base_price * 2.0)

    return df

df_model2 = model2_demand_based(df)

In [15]:
#MODEL3
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c


lot_coords = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].mean().to_dict('index')

def get_nearby_lots(current_lot, max_distance_km=0.5):
    cur = lot_coords[current_lot]
    nearby = []
    for lot, coords in lot_coords.items():
        if lot == current_lot:
            continue
        distance = haversine(cur['Latitude'], cur['Longitude'], coords['Latitude'], coords['Longitude'])
        if distance <= max_distance_km:
            nearby.append(lot)
    return nearby


full_lots = df[df['Occupancy'] >= df['Capacity'] * 0.95]['SystemCodeNumber'].unique()
for lot in full_lots:
    nearby = get_nearby_lots(lot)
    if nearby:
        print(f"\nLot {lot} is full. Consider rerouting to: {nearby}")
    else:
        print(f"\nLot {lot} is full. No nearby alternative.")


Lot BHMBCCMKT01 is full. Consider rerouting to: ['BHMBCCTHL01']

Lot BHMBCCTHL01 is full. Consider rerouting to: ['BHMBCCMKT01']

Lot BHMEURBRD01 is full. No nearby alternative.

Lot BHMMBMMBX01 is full. No nearby alternative.

Lot BHMNCPNST01 is full. Consider rerouting to: ['BHMNCPHST01']

Lot Broad Street is full. No nearby alternative.

Lot Others-CCCPS105a is full. Consider rerouting to: ['Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98']


In [16]:
#Next step
#Real time simulation with pathway
import pathway as pw
class ParkingSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str

# Load data in streaming mode
t = pw.io.csv.read("e3b2e988-6f34-443c-bd4f-eb6fe9616d24.csv",
                  schema=ParkingSchema,
                  mode="streaming",
                  autocommit_duration_ms=500)

# Preprocess and calculate occupancy ratio
t = t.select(
    t.ID,
    t.SystemCodeNumber,
    t.Capacity,
    t.Occupancy,
    t.QueueLength,
    t.VehicleType,
    t.TrafficConditionNearby,
    t.IsSpecialDay,
    occupancy_ratio=t.Occupancy / t.Capacity
)

@pw.udf
def compute_price(base: float, occ_ratio: float) -> float:
    return base + 2.0 * occ_ratio

t = t.with_columns(
    Price=compute_price(10.0, t.occupancy_ratio)
)

# Output result
t = t.select(t.SystemCodeNumber, t.Price, t.occupancy_ratio)
pw.io.jsonlines.write(t, "results_stream.jsonl")

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [19]:
#Visualization using Bokeh
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category10
from bokeh.models import Legend

output_notebook()

p = figure(title="Dynamic Pricing for Parking Lots (Model 1)",
           x_axis_type="datetime",
           width=800, height=400,
           x_axis_label="Time", y_axis_label="Price ($)")

colors = Category10[10] + Category10[4]
legend_items = []

for i, lot in enumerate(df_model1['SystemCodeNumber'].unique()):
    lot_df = df_model1[df_model1['SystemCodeNumber'] == lot]
    line = p.line(x=lot_df['Timestamp'],
                  y=lot_df['Price_Model1'],
                  line_width=2, color=colors[i % len(colors)])
    legend_items.append((lot, [line]))

legend = Legend(items=legend_items, location="center")
p.add_layout(legend, 'right')

show(p)
